In [ ]:
"""
Run script using python scripts/weather_api_call.py in a suitable env
This will update the weather datasets with the latest historical data
"""

from datetime import datetime
import time
import json
from click import format_filename
import pandas as pd
import requests
import pytz
import yaml
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
 def create_5days_list(no_of_days=5):
     a_day = 86400
     day0 = datetime.timestamp(datetime.now())
     past_5_days = []

     for i in range(no_of_days):
         one_day = int(day0) - int(a_day * i)
         past_5_days.append(one_day)
         return past_5_days


past_5_days_list = create_5days_list()

In [14]:
# what is 7pm timestamp BST 26/11/2022
yest_7pm = 1669488240

In [19]:
no_of_days=5

# number of seconds in a day
a_day = 86400

day0 = yest_7pm
past_5_days = []

# for loop to create a list  of timestamps in past 5 days
for i in range(no_of_days):
    one_day = int(day0) - int(a_day * i)
    past_5_days.append(one_day)

past_5_days

[1669488240, 1669401840, 1669315440, 1669229040, 1669142640]

In [ ]:
# def get_historical_weather(
#     lat=51.17,
#     lon=10.45,
#     api_key=data_config['api_key'],
#     past_6_days=past_6_days_list,
# ):
#     """
#     Returns a dataframe with dt and temp columns.

#     Keyword arguments:
#     lat -- the latitude (default 51.17)
#     lon -- the longitude (default 10.45)
#     api_key -- api key gotten from openweathermap
#     past_6_days -- a list of timestamps for the 6 days you want historical data for
 #    """
     new_df = pd.DataFrame()
     past_6_days_df = pd.DataFrame()

     for day in past_6_days:
         url = (
             "https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=%s&lon=%s&dt=%s&units=metric&appid=%s"
             % (lat, lon, day, api_key)
         )
         response = requests.get(url)
         data = json.loads(response.text)
         hourly_data = data["hourly"]

         for entry in hourly_data:
             temp = pd.DataFrame.from_dict(pd.json_normalize(entry), orient="columns")

             # select only nec. columns
             columns = ["dt", "temp"]
             temp = temp[columns]

             # transform dt column
             temp["dt"] = datetime.fromtimestamp(entry["dt"], pytz.utc)
             new_df = new_df.append(temp, ignore_index=True)

         # concatenate each day in a new dataframe
         past_6_days_df = pd.concat(
             [past_6_days_df, new_df], axis=0, ignore_index=False
         ).drop_duplicates()

     return past_6_days_df

lat=51.17
lon=10.45
api_key="bc2d732248597a8725c1f5d51e5d3e8a"
past_5_days=past_5_days

"""
Returns a dataframe with dt and temp columns.

Keyword arguments:
lat -- the latitude (default 51.17)
lon -- the longitude (default 10.45)
api_key -- api key gotten from openweathermap
past_5_days -- a list of timestamps for the 5 days you want historical data for
"""
new_df = pd.DataFrame()
past_5_days_df = pd.DataFrame()

for day in past_5_days:
    url = (
        "https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=%s&lon=%s&dt=%s&units=metric&appid=%s"
        % (lat, lon, day, api_key)
    )
    response = requests.get(url)
    data = json.loads(response.text)
    hourly_data = data["hourly"]

    for entry in hourly_data:
        temp = pd.DataFrame.from_dict(pd.json_normalize(entry), orient="columns")

        # select only nec. columns
        columns = ["dt", "temp"]
        temp = temp[columns]

        # transform dt column
        temp["dt"] = datetime.fromtimestamp(entry["dt"], pytz.utc)
        new_df = new_df.append(temp, ignore_index=True)

    # concatenate each day in a new dataframe
    past_5_days_df = pd.concat(
        [past_5_days_df, new_df], axis=0, ignore_index=False
    ).drop_duplicates()

past_5_days_df

In [ ]:
x=[]
y=np.arange(0,5)
for num in y:
    x1=past_5_days_df['temp'][19+(24*y)]
    x.append(x1)
return x
%matplotlib inline
plt.plot(x,y, 'r*-')
plt.xlabel('temperature')
plt.ylabel('no of days')
plt.title('The tempature at 7pm for the past 5 days')